# 外皮情報LV2からLV3へのコンバート

## 入力情報

### 1) 共通

| 階層1 | 階層2 | 項目名 | 表示条件 | 値:選択要素 | 既定値 | 最小値 | 最大値 | 型 | 小数点以下桁数 | 単位 |
|-------|-------|--------|----------|-------------|--------|--------|--------|----|----------------|------|
| ◯ |   | 地域の区分 |   | 1:1地域<br>2:2地域<br>3:3地域<br>4:4地域<br>5:5地域<br>6:6地域<br>7:7地域<br>8:8地域 | 6 | 1 | 8 | decimal | － | － |
| ◯ |   | その他の居室の有無 |   | 1:yes<br>2:no | 1 | 1 | 2 | decimal |   |   |
| ◯ |   | 非居室の有無 |   | 1:yes<br>2:no | 1 | 1 | 2 | decimal |   |   |
| ◯ |   | 床面積の合計 |   |   | 120.08 | 1.00 | 5000.00 | short | 2 | m<sup>2</sup> |
| ◯ |   | 主たる居室の床面積 |   |   | 29.81 | 1.00 | 5000.00 | short | 2 | m<sup>2</sup> |
| ◯ |   | その他の居室の床面積 |   |   | 51.34 | 1.00 | 5000.00 | short | 2 | m<sup>2</sup> |

### 2) 外皮の部位 (一般部位) (開口部) (複数可)

| 階層1 | 階層2 | 項目名 | 表示条件 | 値:選択要素 | 既定値 | 最小値 | 最大値 | 型 | 小数点以下桁数 | 単位 |
|-------|-------|--------|----------|-------------|--------|--------|--------|----|----------------|------|
| ◯ |   | 名前 |   |   | － | － | － | text | － | － |
| ◯ |   | 面積 |   |　 | ND | 0.000001 | 99.999999 | short | 6 | m<sup>2</sup> |
| ◯ |   | 日射の有無と方位 |   | 1:上面<br>2:北<br>3:北東<br>4:東<br>5:南東<br>6:南<br>7:南西<br>8:西<br>9:北西<br>10:下面<br>11:日射はあたらない | 6 | 1 | 11 | decimal | － | － |
| ◯ |   | 隣接空間の種類 |   | 1:外気・外気に通じる空間<br>2:外気に通じていない空間・外気に通じる床裏<br>3:住戸及び住戸と同様の熱的環境の空間・外気に通じていない床裏 | 1 | 1 | 3 | decimal | － | － |
| ◯ |   | 部位の種類 |   | 1:一般部位(木造)<br>2:一般部位(鉄骨造)<br>3:一般部位(鉄筋コンクリート造)<br>4:窓等の大部分がガラスで構成されている開口部<br>5:ドア等の大部分がガラスで構成されていない開口部 | 1 | 1 | 5 | short | － | － |
|   | ◯ | 部位の位置 | [部位の種類]=1,2,3 | 1:屋根<br>2:天井<br>3:外壁<br>4:床<br>5:界壁<br>6:界床<br>7:天井(界床) | ND | 1 | 7 | short | － | － |
| ◯ |   | 熱貫流率<sup>※1</sup> |   |   | ND | ND | ND | short | ND | W/m<sup>2</sup>K |
| ◯ |   | 暖房期の日射熱取得率<sup>※1</sup> |   |   | ND | ND | ND | short | ND | － |
| ◯ |   | 冷房期の日射熱取得率<sup>※1</sup> |   |　 | ND | ND | ND | short | ND | － |

※1 本項目は直接入力される値ではないため、最小値・最大値・小数点以下桁数等は定めない。

### 3) 外皮の部位 (熱橋等) (複数可)

| 階層1 | 階層2 | 項目名 | 表示条件 | 値:選択要素 | 既定値 | 最小値 | 最大値 | 型 | 小数点以下桁数 | 単位 |
|-------|-------|--------|----------|-------------|--------|--------|--------|----|----------------|------|
| ◯ |   | 名前 |   |   | － | － | － | text | － | － |
| ◯ |   | 長さ |   |   | ND | 0.001 | 99.999 | short | 3 | m |
| ◯ |   | 日射の有無と方位1  |   | 1:上面<br>2:北<br>3:北東<br>4:東<br>5:南東<br>6:南<br>7:南西<br>8:西<br>9:北西<br>10:下面<br>11:日射はあたらない | 6 | 1 | 11 | decimal | － | － |
| ◯ |   | 日射の有無と方位2  |   | 1:上面<br>2:北<br>3:北東<br>4:東<br>5:南東<br>6:南<br>7:南西<br>8:西<br>9:北西<br>10:下面<br>11:日射はあたらない | 6 | 1 | 11 | decimal | － | － |
| ◯ |   | 隣接空間の種類 |   | 1:外気・外気に通じる空間<br>2:外気に通じていない空間・外気に通じる床裏<br>3:住戸及び住戸と同様の熱的環境の空間・外気に通じていない床裏 | 1 | 1 | 3 | decimal | － | － |
| ◯ |   | 部位の種類 |   | 1:熱橋<br>2:土間床等の外周部 | 1 | 1 | 2 | short | － | － |
| ◯ |   | 線熱貫流率<sup>※1</sup> |   |   | ND | ND | ND | short | ND | W/mK |
| ◯ |   | 暖房期の線日射熱取得率<sup>※1</sup> |   |   | ND | ND | ND | short | ND | (W/m<sup>2</sup>)/(W/m<sup>2</sup>) |
| ◯ |   | 冷房期の線日射熱取得率<sup>※1</sup> |   |   | ND | ND | ND | short | ND | (W/m<sup>2</sup>)/(W/m<sup>2</sup>) |

※1 本項目は直接入力される値ではないため、最小値・最大値・小数点以下桁数等は定めない。

### 4) 外皮の部位 (土間床等の中央部) (複数可)

| 階層1 | 階層2 | 項目名 | 表示条件 | 値:選択要素 | 既定値 | 最小値 | 最大値 | 型 | 小数点以下桁数 | 単位 |
|-------|-------|--------|----------|-------------|--------|--------|--------|----|----------------|------|
| ◯ |   | 名前 |   |   | － | － | － | text | － | － |
| ◯ |   | 面積 |   |   | ND | 0.000001 | 99.999999 | short | 6 | m<sup>2</sup> |
| ◯ |   | (断熱性能を表す指標)<sup>※1</sup> |   |   | ND | ND | ND | ND | ND | － |

※1 本項目は直接入力される値ではないため、最小値・最大値・小数点以下桁数等は定めない。

## 出力情報

### 共通

| 階層1 | 階層2 | 項目名 | 表示条件 | 値:選択要素 | 既定値 | 最小値 | 最大値 | 型 | 小数点以下桁数 | 単位 |
|-------|-------|--------|----------|-------------|--------|--------|--------|----|----------------|------|
| ◯ |   | 地域の区分 |   | 1:1地域<br>2:2地域<br>3:3地域<br>4:4地域<br>5:5地域<br>6:6地域<br>7:7地域<br>8:8地域 | 6 | 1 | 8 | decimal | － | － |
| ◯ |   | その他の居室の有無 |   | 1:yes<br>2:no | 1 | 1 | 2 | decimal |   |   |
| ◯ |   | 非居室の有無 |   | 1:yes<br>2:no | 1 | 1 | 2 | decimal |   |   |
| ◯ |   | 床面積の合計 |   |   | 120.08 | 1.00 | 5000.00 | short | 2 | m<sup>2</sup> |
| ◯ |   | 主たる居室の床面積 |   |   | 29.81 | 1.00 | 5000.00 | short | 2 | m<sup>2</sup> |
| ◯ |   | その他の居室の床面積 |   |   | 51.34 | 1.00 | 5000.00 | short | 2 | m<sup>2</sup> |

### 外皮の部位 (一般部位) (開口部) (複数可)

| 階層1 | 階層2 | 項目名 | 表示条件 | 値:選択要素 | 既定値 | 最小値 | 最大値 | 型 | 小数点以下桁数 | 単位 |
|-------|-------|--------|----------|-------------|--------|--------|--------|----|----------------|------|
| ◯ |   | 名前 |   |   | － | － | － | text | － | － |
| ◯ |   | 面積 |   |　 | ND | 0.000001 | 99.999999 | short | 6 | m<sup>2</sup> |
| ◯ |   | 室内側の空間の用途 |   | 1:主たる居室<br>2:その他の居室<br>3:非居室<br>4:床下空間 | ND | 1 | 4 | short | － | － |
| ◯ |   | 日射の有無と方位 |   | 1:上面<br>2:北<br>3:北東<br>4:東<br>5:南東<br>6:南<br>7:南西<br>8:西<br>9:北西<br>10:下面<br>11:日射はあたらない | 6 | 1 | 11 | decimal | － | － |
| ◯ |   | 隣接空間の種類 |   | 1:外気・外気に通じる空間<br>2:外気に通じていない空間・外気に通じる床裏<br>3:住戸及び住戸と同様の熱的環境の空間・外気に通じていない床裏 | 1 | 1 | 3 | decimal | － | － |
| ◯ |   | 部位の種類 |   | 1:一般部位(木造)<br>2:一般部位(鉄骨造)<br>3:一般部位(鉄筋コンクリート造)<br>4:窓等の大部分がガラスで構成されている開口部<br>5:ドア等の大部分がガラスで構成されていない開口部 | 1 | 1 | 5 | short | － | － |
|   | ◯ | 部位の位置 | [部位の種類]=1,2,3 | 1:屋根<br>2:天井<br>3:外壁<br>4:床<br>5:界壁<br>6:界床<br>7:天井(界床) | ND | 1 | 7 | short | － | － |
| ◯ |   | 熱貫流率<sup>※1</sup> |   |   | ND | ND | ND | short | ND | W/m<sup>2</sup>K |
| ◯ |   | 暖房期の日射熱取得率<sup>※1</sup> |   |   | ND | ND | ND | short | ND | － |
| ◯ |   | 冷房期の日射熱取得率<sup>※1</sup> |   |　 | ND | ND | ND | short | ND | － |

※1 本項目は直接入力される値ではないため、最小値・最大値・小数点以下桁数等は定めない。

### 外皮の部位 (熱橋等) (複数可)

| 階層1 | 階層2 | 項目名 | 表示条件 | 値:選択要素 | 既定値 | 最小値 | 最大値 | 型 | 小数点以下桁数 | 単位 |
|-------|-------|--------|----------|-------------|--------|--------|--------|----|----------------|------|
| ◯ |   | 名前 |   |   | － | － | － | text | － | － |
| ◯ |   | 長さ |   |   | ND | 0.001 | 99.999 | short | 3 | m |
| ◯ |   | 室内側の空間の用途 |   | 1:主たる居室<br>2:その他の居室<br>3:非居室<br>4:床下空間 | ND | 1 | 4 | short | － | － |
| ◯ |   | 日射の有無と方位1  |   | 1:上面<br>2:北<br>3:北東<br>4:東<br>5:南東<br>6:南<br>7:南西<br>8:西<br>9:北西<br>10:下面<br>11:日射はあたらない | 6 | 1 | 11 | decimal | － | － |
| ◯ |   | 日射の有無と方位2  |   | 1:上面<br>2:北<br>3:北東<br>4:東<br>5:南東<br>6:南<br>7:南西<br>8:西<br>9:北西<br>10:下面<br>11:日射はあたらない | 6 | 1 | 11 | decimal | － | － |
| ◯ |   | 隣接空間の種類 |   | 1:外気・外気に通じる空間<br>2:外気に通じていない空間・外気に通じる床裏<br>3:住戸及び住戸と同様の熱的環境の空間・外気に通じていない床裏 | 1 | 1 | 3 | decimal | － | － |
| ◯ |   | 部位の種類 |   | 1:熱橋<br>2:土間床等の外周部 | 1 | 1 | 2 | short | － | － |
| ◯ |   | 線熱貫流率<sup>※1</sup> |   |   | ND | ND | ND | short | ND | W/mK |
| ◯ |   | 暖房期の線日射熱取得率<sup>※1</sup> |   |   | ND | ND | ND | short | ND | (W/m<sup>2</sup>)/(W/m<sup>2</sup>) |
| ◯ |   | 冷房期の線日射熱取得率<sup>※1</sup> |   |   | ND | ND | ND | short | ND | (W/m<sup>2</sup>)/(W/m<sup>2</sup>) |

※1 本項目は直接入力される値ではないため、最小値・最大値・小数点以下桁数等は定めない。

### 外皮の部位 (土間床等の中央部) (複数可)

| 階層1 | 階層2 | 項目名 | 表示条件 | 値:選択要素 | 既定値 | 最小値 | 最大値 | 型 | 小数点以下桁数 | 単位 |
|-------|-------|--------|----------|-------------|--------|--------|--------|----|----------------|------|
| ◯ |   | 名前 |   |   | － | － | － | text | － | － |
| ◯ |   | 面積 |   |   | ND | 0.000001 | 99.999999 | short | 6 | m<sup>2</sup> |
| ◯ |   | 室内側の空間の用途 |   | 1:主たる居室<br>2:その他の居室<br>3:非居室<br>4:床下空間 | ND | 1 | 4 | short | － | － |
| ◯ |   | (断熱性能を表す指標)<sup>※1</sup> |   |   | ND | ND | ND | ND | ND | － |

※1 本項目は直接入力される値ではないため、最小値・最大値・小数点以下桁数等は定めない。

## 2. 外皮性能計算プログラム＋簡単な追加情報

- 外皮性能計算プログラムの入力情報に、用途別床面積を追加したものより、推定を行う。
- 現状の外皮性能計算プログラムで入力されるのは用途別に分かれていない外皮性能であるため、室用途別の性能を求めるためには、外皮を用途別に割り振る必要がある。
- 住戸全体の各方位の外皮は、室用途別の床面積に応じて各用途に割り当てる。集合住宅の戸境壁等も同様の扱いとする。外皮性能は用途によらず均一に割り付ける。開口部は、2.外皮性能計算プログラムの入力情報より推定を開始する場合は、庇等の日射遮蔽部材を考慮した計算を行うため、方位、遮蔽部材やその寸法等が異なる場合は、別部材として扱う。その他は3.と同様に求める。
 
$$ \qquad
A_{主,i,j}= \frac{A_{i,j}・A_主}{A_主 + A_他 + A_非}, \quad
A_{他,i,j}= \frac{A_{i,j}・A_他}{A_主 + A_他 + A_非}\quad, \quad
A_{非,i,j}= \frac{A_{i,j}・A_主}{A_非 + A_他 + A_非}\quad
\qquad (1),\;(2),\;(3)
\\
$$
$$ \qquad
L_{主,i,j}= \frac{L_{i,j}・A_主}{A_主 + A_他 + A_非}, \quad
L_{他,i,j}= \frac{L_{i,j}・A_他}{A_主 + A_他 + A_非}\quad, \quad
L_{非,i,j}= \frac{L_{i,j}・A_主}{A_非 + A_他 + A_非}\quad
\qquad (4),\;(5),\;(6)
\\
$$
$$ \qquad
U_{主,i,j} =U_{他,i,j} =U_{非,i,j} =U_{i,j} 
\qquad (7)
\\
$$
$$ \qquad
Ψ_{主,i,j} =Ψ_{他,i,j} =Ψ_{非,i,j} =Ψ_{i,j} 
\qquad (8)
\\
$$
$$ \qquad
η_{主,i,j} =η_{他,i,j} =η_{非,i,j} =η_{i,j} 
\qquad (8)
\\
$$

- 入力値：

  - 主居室、その他の居室、非居室の床面積$[m ^ 2]$：$A_主$、$A_他$、$A_非$
  - 外皮の方位j、部位（一般部位又は開口部）𝑖の面積$[m ^ 2]$：$A_{i,j}$
  - 熱橋及び土間床等の外周部jの長さ$[m]$：$L_{i,j}$
  - 外皮の方位j、部位（一般部位又は開口部）iの熱貫流率$[W/m ^ 2 K]$：$U_{i,j}$
  - 熱橋及び土間床等の外周部jの線熱貫流率$[W/mK]$：$\psi _i$$_,$$_j$
  - 外皮の方位j、部位（一般部位又は開口部）iの日射熱取得率$[-]$：$\eta_{i,j}$

- 出力値：

  - 主居室、その他の居室、非居室の外皮の方位j、部位（一般部位又は開口部）iの面積$[m ^ 2]$：$A_{主,i,j}$、$A_{他,i,j}$、$A_{非,i,j}$
  - 主居室、その他の居室、非居室の：熱橋及び土間床等の方位jの外周部iの長さ$[m]$：$L_{主,i,j}$、$L_{他,i,j}$、$L_{非,i,j}$
  - 主居室、その他の居室、非居室の外皮の方位j、部位（一般部位又は開口部）iの熱貫流率$[W/m ^ 2 K]$：$U_{主,i,j}$、$U_{他,i,j}$、$U_{非,i,j}$   
  - 主居室、その他の居室、非居室の熱橋及び土間床等の方位jの外周部iの線熱貫流率$[W/mK]$：$\psi _{主,i,j}$、$\psi _{他,i,j}$、$\psi_{非,i,j}$
  - 主居室、その他の居室、非居室の外皮の方位j、部位（一般部位又は開口部）iの日射熱取得率$[-]$：$\eta _{主,i,j}$、$\eta _{他,i,j}$、$\eta_{非,i,j}$

In [14]:
#地域の区分 (1~8)
def Calc_Rg3(Rg2):
    Rg3 =  Rg2
    return Rg3

Rg2=6
print ("Rg3={0}".format(Calc_Rg3(Rg2)))

Rg3=6


In [11]:
#その他の居室の有無 (yes/no)
#非居室の有無 (yes/no)
def Calc_Flag3(Flag2):
    Flag3 =  Flag2
    return Flag3

Flag22=1
print ("Flag23={0}".format(Calc_Flag3(Flag22)))

Flag32=1
print ("Flag33={0}".format(Calc_Flag3(Flag32)))

Flag23=1
Flag33=1


In [13]:
#床面積の合計 (m2)
#主たる居室の床面積 (m2)
#その他の居室の床面積 (m2)
def Calc_A3(A2):
    A3 =  A2
    return A3

Aa2=120
print ("Aa3={0}".format(Calc_A3(Aa2)))

A12=30
print ("A13={0}".format(Calc_A3(A12)))

A22=50
print ("A23={0}".format(Calc_A3(A22)))

Aa3=120
A13=30
A23=50


In [1]:
""" 室用途別の面積の算出 """
def Calc_Apu(Ap, Au, Aa):
    Apu =  Ap*Au/Aa
    return Apu

print (Calc_Apu(1, 1, 5))

0.2


In [2]:
""" 室用途別の長さの算出 """
def Calc_Lpu(Lp, Au, Aa):
    Lpu =  Lp*Au/Aa
    return Lpu

print (Calc_Lpu(1, 1, 5))

0.2


In [3]:
""" 室用途別の熱貫流率の算出 """
def Calc_Upu(Up):
    Upu =  Up
    return Upu

print (Calc_Upu(1))

1


In [4]:
""" 室用途別の線熱貫流率の算出 """
def Calc_Ψpu(Ψp):
    Ψpu =  Ψp
    return Ψpu

print (Calc_Ψpu(1))

1


In [15]:
""" 室用途別の日射熱取得率の算出 """
def Calc_ηpu(ηp):
    ηpu =  ηp
    return ηpu

print (Calc_ηpu(1))

1
